
1. 数据脱敏，将违章者的姓名、驾驶证号等个人隐私信息替换；
2. 交通违章行为危害性评估：根据违章的类型、描述、事故等级等信息，给每次违章设置一个危害指数；
3. 提取违章者的违法行为特征变量l1,l2,l3…如司机在一年内违章的次数、两次违章之间间隔、违章危害指数等；
4. 对违章者数据进行聚类：对3中的特征变量进行归一化、划分等级、以及相关性评估，选取用于聚类的变量，利用k-means进行聚类。得到k个违章者群体。
5. 分别对k个违章群体进行描述性分析：属于这个群体的违章者的主要准驾车辆、违章时间分布、主要违章行为等信息，词云可视化；
6. 选取白天/夜间，工作日/周末等研究区间，进行4-5步骤，探究在不同的时间段内违章群体的构成与特征是否会发生变化。

In [2]:
import pandas as pd
from IPython.display import display
import numpy as np

from matplotlib import pyplot as plt

In [3]:
# data_path = "/exstorage/zjr/traffic/data/data3.csv"
data_path1 = "/Users/jiarui/Study/交通事故/data/海宁2020.1-6.csv"
data_path2 = "/Users/jiarui/Study/交通事故/data/海宁2020.7-12.csv"
data_path2019 = ""
data_path2018 =

df1 = pd.read_csv(data_path1)
df2 = pd.read_csv(data_path2)
print(len(df1))
print(len(df2))

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7,36,38,39,40,43,44,48,49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (38,39,40,43,44,48,49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


273732
362170


In [4]:
df = pd.concat([df1,df2])
print(len(df))

635902


## 数据预处理
数据脱敏

选择有需要的列

筛选出机动车和非机动车，机动车可以根据驾驶证号计算年龄

In [8]:
## 选择有需要的列
df = df[['违法时间', '当事人_驾驶证号',
         '当事人_发证机关', '当事人_准驾车型', '机动车_号牌种类', '机动车_号牌种类说明', '机动车_使用性质说明', '机动车_号牌号码', '机动车_交通方式',
         '机动车_交通方式说明', '违法地点', '违法地点_道路类型', '违法地点_路段代码', '违法地点_地点米数',
         '违法地点_违法地址', '违法地点_公路行政等级', '违法行为', '违法行为_违法大类', '违法行为_违法小类',
         '违法行为_违法描述', '违法行为_事故等级', '违法行为记分数', '违法行为罚款金额', '违法行为_实测值', '违法行为_标准值']]

In [9]:
list_display = ['当事人_驾驶证号', '违法时间', '机动车_交通方式说明', '违法行为_违法小类', '违法行为_违法描述', '违法行为记分数', '违法行为罚款金额']

In [10]:
## 筛选机动车与非机动车
pd.set_option('display.max_rows', 200)
action_count = df['机动车_交通方式说明'].value_counts()
# print(action_count.head(104))
print(len(action_count))

# action_count = df['机动车_号牌种类说明'].value_counts()
# print(action_count.head(104))
# print(len(action_count))
list_no_vehicle = ['步行', '乘车', '电动自行车', '其他非机动车', '自行车', '三轮车']
df_no_vehicle = df[df['机动车_交通方式说明'].isin(list_no_vehicle)]
df_vehicle = df.drop(df[df['机动车_交通方式说明'].isin(list_no_vehicle)].index)

action_count = df_no_vehicle['机动车_交通方式说明'].value_counts()
print(action_count.head(104))
print(len(action_count))

action_count = df_vehicle['机动车_交通方式说明'].value_counts()
print(action_count.head(104))
print(len(action_count))

action_count = df_vehicle['机动车_交通方式'].value_counts()
print(action_count.head(104))
print(len(action_count))

87
电动自行车     135361
步行          4512
其他非机动车      3933
乘车          1023
三轮车          520
自行车          132
Name: 机动车_交通方式说明, dtype: int64
6
小型轿车           237513
小型普通客车          99288
小型越野客车          17699
小型面包车           13419
轻型栏板货车           9269
重型半挂牵引车          6528
轻型厢式货车           6358
重型自卸货车           3957
重型厢式货车           2964
重型栏板货车           2842
重型仓栅式货车          2772
普通二轮摩托车          2539
轻型自卸货车           1171
轻型封闭式货车          1094
正三轮载货摩托车          897
轻型仓栅式货车           743
重型特殊结构货车          704
中型栏板货车            546
微型轿车              447
重型集装箱半挂车          445
重型自卸半挂车           306
中型厢式货车            305
重型非载货专项作业车        262
中型普通客车            215
重型罐式货车            213
大型普通客车            210
重型栏板半挂车           207
轻便二轮摩托车           170
重型平板半挂车           157
重型平板自卸半挂车         144
重型平板货车            140
重型低平板半挂车          130
小型专用客车            121
轻型非载货专项作业车        117
重型罐式半挂车            82
轻型多用途货车            77
重型仓栅式半挂车           67
中型非载货专项作业车         49
中型仓栅式货车            46
中型自卸

In [11]:
# 区分营运车辆和非营运车辆
df_vehicle['机动车_使用性质说明'].value_counts()

非营运       382573
货运         24247
预约出租客运      3723
营转非         1996
出租客运         701
教练           468
租赁           358
危化品运输        264
公交客运         109
预约出租转非        84
旅游客运          69
公路客运          29
工程救险          24
出租转非          16
救护             4
警用             1
Name: 机动车_使用性质说明, dtype: int64

In [12]:
# print(df_vehicle[df_vehicle['机动车_使用性质说明'] == '非营运']['机动车_交通方式说明'].value_counts())

# print(df_vehicle[df_vehicle['机动车_使用性质说明'] == '租赁']['机动车_交通方式说明'].value_counts())
df_no_yingyun = df_vehicle[df_vehicle['机动车_使用性质说明'] == '非营运']
df_yingyun = df_vehicle[df_vehicle['机动车_使用性质说明'] != '非营运']

In [13]:
# 计算dataframe的各个维度特征，如交通方式编码、计算年龄、给违法行为评估
def data_get_feature(df_vehicle):
    ## 直接去掉没有驾驶证号的
    print('原来机动车违法的数据有{}条'.format(len(df_vehicle)))
    df_vehicle = df_vehicle.drop(df_vehicle[df_vehicle['当事人_驾驶证号'].isin(['无', np.nan])].index)
    # 删除掉驾驶证号不完全或者有误的
    df_vehicle = df_vehicle.loc[df_vehicle['当事人_驾驶证号'].str.len() == 18].reset_index()
    print('经过去除无驾驶证号的之后，还有{}'.format(len(df_vehicle)))
    df_vehicle[df_vehicle['当事人_驾驶证号'].isin([np.nan])][['当事人_驾驶证号',
                                                       '当事人_发证机关', '当事人_准驾车型', '机动车_号牌种类', '机动车_号牌种类说明', '机动车_交通方式',
                                                       '机动车_交通方式说明', '违法地点']]

    # 给交通方式编码
    df_vehicle['trans_code'] = 0
    # print(df_vehicle['机动车_交通方式说明'].str.contains('小型轿车'))
    list_1 = ['小型轿车', '面包车']
    df_vehicle.loc[df_vehicle['机动车_交通方式说明'].str.contains('小型轿车|面包车'), 'trans_code'] = 1

    list_2 = ['货车', '大型轿车', '作业车', '大型专用校车']
    df_vehicle.loc[df_vehicle['机动车_交通方式说明'].str.contains('货车|大型轿车|作业车|大型专用校车|有轨电车|手扶变形运输机'), 'trans_code'] = 2

    list_3 = ['牵引车', '挂车']
    df_vehicle.loc[df_vehicle['机动车_交通方式说明'].str.contains('牵引车|挂车|轮式挖掘机械'), 'trans_code'] = 3

    list_4 = ['客车']
    df_vehicle.loc[df_vehicle['机动车_交通方式说明'].str.contains('客车'), 'trans_code'] = 4

    list_5 = ['摩托车']
    df_vehicle.loc[df_vehicle['机动车_交通方式说明'].str.contains('摩托车'), 'trans_code'] = 5

    df_vehicle.loc[df_vehicle['机动车_交通方式说明'].str.contains('微型轿车|三轮汽车|残疾人专用车|助力自行车|手推车|其它|拖拉机'), 'trans_code'] = 6

    print("--------------交通方式编码完成----------------")

    ## 计算年龄
    born_year = df_vehicle['当事人_驾驶证号'].str[6:10]
    # born_year = born_year.astype('int64'，errors = 'ignore')
    # pd.to_numeric(born_year, errors='coerce').fillna(0)
    df_vehicle['age'] = 2021 - pd.to_numeric(born_year, errors='coerce').fillna(0)
    df_vehicle = df_vehicle.drop(
        df_vehicle[(df_vehicle['age'] > 80) | (df_vehicle['age'] < 10) | (df_vehicle['age'] == np.nan)].index)

    # 筛选违法行为类型
    df_vehicle = df_vehicle.drop(
        df_vehicle[(df_vehicle['违法行为_违法小类'] == '其他影响安全行为') | (df_vehicle['违法行为_违法小类'] == '其他违法行为')].index)
    df_vehicle = df_vehicle.drop(
        df_vehicle[(df_vehicle['违法行为_违法小类'] == '其他') | (df_vehicle['违法行为_违法小类'] == '擅自销售生产机动车')].index)
    df_vehicle = df_vehicle.drop(
        df_vehicle[(df_vehicle['违法行为_违法小类'] == '铁路道口或渡口') | (df_vehicle['违法行为_违法小类'] == '违法占道')].index)
    df_vehicle = df_vehicle.drop(
        df_vehicle[(df_vehicle['违法行为_违法小类'] == '违法牵引') | (df_vehicle['违法行为_违法小类'] == '非法获取机动车牌证')].index)
    df_vehicle.loc[(df_vehicle['违法行为_违法小类'] == '违反交通信号') & (
        df_vehicle['违法行为_违法描述'].str.contains('违反道路交通信号灯')), '违法行为_违法小类'] = '违反交通信号灯'

    print(len(df_vehicle))
    #     print(df_vehicle['违法行为_违法小类'].value_counts())

    #给违法行为的危险性打分
    df_vehicle['occur_prob'] = 1.0
    df_vehicle['severity'] = 1.0

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法上道路行驶', 'occur_prob'] = 1
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法上道路行驶', 'severity'] = 1

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法停车', 'occur_prob'] = 1
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法停车', 'severity'] = 1

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违反交通信号', 'occur_prob'] = 1
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违反交通信号', 'severity'] = 1
    df_vehicle.loc[
        (df_vehicle['违法行为_违法小类'] == '违反交通信号') & (df_vehicle['违法行为_违法描述'].str.contains('违反交通信号')), 'occur_prob'] = 5
    df_vehicle.loc[
        (df_vehicle['违法行为_违法小类'] == '违反交通信号') & (df_vehicle['违法行为_违法描述'].str.contains('违反交通信号')), 'severity'] = 5
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违反交通信号灯', 'severity'] = 5
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违反交通信号灯', 'occur_prob'] = 5

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法装载', 'occur_prob'] = 2
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法装载', 'severity'] = 5

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '未按规定办理业务', 'occur_prob'] = 1
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '未按规定办理业务', 'severity'] = 1

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法占道行驶', 'occur_prob'] = 3
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法占道行驶', 'severity'] = 3

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法抢行', 'occur_prob'] = 5
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法抢行', 'severity'] = 4

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '货动车辆超载', 'occur_prob'] = 2
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '货动车辆超载', 'severity'] = 5

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法装载', 'occur_prob'] = 2
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法装载', 'severity'] = 5

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '不按规定使用灯光', 'occur_prob'] = 2
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '不按规定使用灯光', 'severity'] = 3

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '饮酒驾驶', 'occur_prob'] = 5
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '饮酒驾驶', 'severity'] = 5

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '超速行驶', 'occur_prob'] = 5
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '超速行驶', 'severity'] = 4

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法掉头', 'occur_prob'] = 4
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法掉头', 'severity'] = 4

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '疲劳驾驶', 'occur_prob'] = 5
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '疲劳驾驶', 'severity'] = 5

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '无证驾驶', 'occur_prob'] = 4
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '无证驾驶', 'severity'] = 3

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '逆行', 'occur_prob'] = 5
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '逆行', 'severity'] = 4

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '未按规定让行', 'occur_prob'] = 1
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '未按规定让行', 'severity'] = 1

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法倒车', 'occur_prob'] = 2
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法倒车', 'severity'] = 2

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '醉酒驾驶', 'occur_prob'] = 5
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '醉酒驾驶', 'severity'] = 5

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法变更车道', 'occur_prob'] = 5
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法变更车道', 'severity'] = 4

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '客运车辆超员', 'occur_prob'] = 2
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '客运车辆超员', 'severity'] = 5

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '未低速通过', 'occur_prob'] = 4
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '未低速通过', 'severity'] = 4

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法超车', 'occur_prob'] = 4
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法超车', 'severity'] = 3

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法装载超限及危险品运输', 'occur_prob'] = 2
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法装载超限及危险品运输', 'severity'] = 5

    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法会车', 'occur_prob'] = 5
    df_vehicle.loc[df_vehicle['违法行为_违法小类'] == '违法会车', 'severity'] = 4

    # 外来或本地,省内为1 省外为2
    df_vehicle['is_local'] = 1
    df_vehicle.loc[df_vehicle['机动车_号牌号码'].str[0:1] != '浙', 'is_local'] = 2
    df_vehicle['is_local'].value_counts()

    #数据概况
    driver_count = df_vehicle['当事人_驾驶证号'].value_counts()
    count = (driver_count >= 3).sum()
    count2 = (driver_count == 2).sum()
    print(
        "机动车总共违法记录{}，总共有{}个违法人\n其中违法次数大于等于3的有{}人\n违法次数2的有{}人".format(len(df_vehicle), len(driver_count), count, count2))

    return df_vehicle

In [14]:
df_no_yingyun_value = data_get_feature(df_no_yingyun)
df_yingyun_value = data_get_feature(df_yingyun)

原来机动车违法的数据有382573条
经过去除无驾驶证号的之后，还有336616
--------------交通方式编码完成----------------
313886
机动车总共违法记录313886，总共有222036个违法人
其中违法次数大于等于3的有19871人
违法次数2的有40039人
原来机动车违法的数据有32106条
经过去除无驾驶证号的之后，还有29936
--------------交通方式编码完成----------------
28070
机动车总共违法记录28070，总共有21682个违法人
其中违法次数大于等于3的有1412人
违法次数2的有2620人


In [15]:
df_yingyun_value.to_csv("/Users/jiarui/Study/交通事故/data/output/df_yingyun_value.csv")
df_no_yingyun_value.to_csv("/Users/jiarui/Study/交通事故/data/output/df_no_yingyun_value.csv")

### 选取聚类的特征变量

基本属性：性别、本地/外来，驾驶车型、年龄、驾龄；

行为属性：个人在一年内的违章次数、罚款金额、总扣分数、违章危害得分等；

In [16]:
df_driver_info = pd.read_csv("/Users/jiarui/Study/交通事故/data/海宁驾驶证信息表.csv")
def get_level(df_vehicle):
    df_calculate = df_vehicle[
        ['当事人_驾驶证号', 'age', 'is_local', 'trans_code', 'occur_prob', 'severity', '违法行为记分数', '违法行为罚款金额', '违法行为_事故等级']]
    # df_res是每个司机的数值，df_level是每个司机数值划分后的结果
    df_res = df_calculate.groupby('当事人_驾驶证号').agg('mean')
    df_res.columns = ['age', 'is_local', 'trans_code', 'occur_prob', 'severity', 'score', 'fine', 'accident']
    df_res.index_name = '当事人_驾驶证号'
    df_res['counts'] = df_calculate['当事人_驾驶证号'].value_counts()
    for uid in df_calculate['当事人_驾驶证号']:
        dfi = df_driver_info[df_driver_info["身份证明号码"] == uid]
        gender = dfi["性别"]
        first_year = dfi['初次领证日期'].str[6:10]
        drive_age = 2021 - pd.to_numeric(first_year, errors='coerce').fillna(0)
        driver_source = dfi["驾驶人来源说明"]
        driver_address = dfi["联系住所详细地址"]
        df_res[df_res['当事人_驾驶证号' == uid]]['gender'] = gender
        df_res[df_res['当事人_驾驶证号' == uid]]['driver_age'] = drive_age
        df_res[df_res['当事人_驾驶证号' == uid]]['source'] = driver_source
        df_res[df_res['当事人_驾驶证号' == uid]]['address'] = driver_address

    #     df_res[df_res['counts'] < 20]['counts'].plot(kind='hist')
    # 如果只是df_level=df_res 是浅拷贝，和引用类似，会导致修改原来的值
    df_level = df_res.copy(deep=True)
    df_level = df_level.round(0)
    list_col = df_res.columns
    # list_col = ['counts']
    for col in list_col:
        if col == 'id' or col == 'trans_code' or col == 'is_local':
            continue
        dfi = df_level[col].copy(deep=True)
        Q1 = df_level[col].quantile(0.25)
        Q2 = df_level[col].quantile(0.5)
        Q3 = df_level[col].quantile(0.75)
        IQR = Q3 - Q1
        Max = Q3 + IQR * 1.5
        Min = Q1 - IQR * 1.5
        if col == 'counts':
            df_level.loc[dfi >= 5, col] = 5
        elif col == 'score':
            df_level.loc[dfi <= 0.5, col] = 1
            df_level.loc[(dfi < 1) & (dfi > 0.5), col] = 2
            df_level.loc[(dfi < 2) & (dfi >= 1), col] = 3
            df_level.loc[(dfi < 3) & (dfi >= 2), col] = 4
            df_level.loc[(dfi < 6) & (dfi >= 3), col] = 5
            df_level.loc[dfi >= 6, col] = 6
        elif col == 'fine':
            df_level.loc[dfi < 50, col] = 1
            df_level.loc[(dfi < 100) & (dfi >= 50), col] = 2
            df_level.loc[(dfi < 150) & (dfi >= 100), col] = 3
            df_level.loc[(dfi < 200) & (dfi >= 150), col] = 4
            df_level.loc[(dfi < 300) & (dfi >= 200), col] = 5
            df_level.loc[dfi >= 300, col] = 6
        elif col == 'age':
            df_level.loc[dfi <= 25, col] = 1
            df_level.loc[(dfi < 35) & (dfi >= 25), col] = 2
            df_level.loc[(dfi < 45) & (dfi >= 35), col] = 3
            df_level.loc[(dfi < 55) & (dfi >= 45), col] = 4
            df_level.loc[dfi >= 55, col] = 5
        elif col == 'accident':
            df_level.loc[dfi == 0, col] = 1
            df_level.loc[(dfi <= 2) & (dfi > 0), col] = 2
            df_level.loc[(dfi <= 4) & (dfi > 2), col] = 3
        # else:
        #     df_level.loc[(dfi < Min) & (dfi >= 0), col] = 1
        #     df_level.loc[(dfi < Q1) & (dfi >= Min), col] = 2
        #     df_level.loc[(dfi < Q2) & (dfi >= Q1), col] = 3
        #     df_level.loc[(dfi < Q3) & (dfi >= Q2), col] = 4
        #     df_level.loc[(dfi < Max) & (dfi >= Q3), col] = 5
        #     df_level.loc[dfi >= Max, col] = 6
    df_level.index.name = 'id'
    display(df_level)
    return df_level

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11,18,19,46) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
print("--------非营运车辆--------")
df_no_yingyun_level = get_level(df_no_yingyun_value)
print("--------营运车辆--------")
df_yingyun_level = get_level(df_yingyun_value)

--------非营运车辆--------


KeyError: False

In [ ]:
df_no_yingyun_level.loc[df_no_yingyun_level.isin([np.nan]).any(axis=1), 'fine'] = 0
df_no_yingyun_level[df_no_yingyun_level.isin([np.nan]).any(axis=1)]

In [ ]:
# 数据概况
import matplotlib.pyplot as plt

fig = plt.figure()
fig.tight_layout()


# 输入的是一个df，主键是每个司机的驾驶证号id
# 得到这个司机群体的信息
def analyseDF(df_input):
    plt.figure(figsize=(40, 40))

    # 年龄组成
    ages = df_input['age'].value_counts()
    plt.subplot(4, 1, 1)
    df_input['age'].plot(kind='hist')
    plt.title("Count of age")

    # print(ages)
    # 外地or本地
    is_local = df_input['is_local'].value_counts()
    print(is_local)
    # 准驾车型
    trans_codes = df_input['机动车_交通方式说明'].value_counts()
    print(trans_codes)

    # 违法行为记分数
    scores = df_input['违法行为记分数'].value_counts()
    print(scores)

    type_counts = df_input['违法行为_违法小类'].value_counts()
    print(type_counts.head(10))
    plt.subplot(4, 1, 2)
    word_cloud(type_counts)

    detail_count = df_input['违法行为_违法描述'].value_counts()
    print(detail_count.head(10))
    plt.subplot(4, 1, 3)
    word_cloud(detail_count)

    # 计算违章时间段统计

    df_input['违法时间'] = pd.to_datetime(df_input['违法时间'])
    df_input['违法时间'] = pd.to_datetime(df_input['违法时间'])
    df_date = df_input.sort_values('违法时间')
    df_date = df_date.set_index('违法时间')
    # print(df_date.truncate(before='2019',after='2022-1').head())
    df_date_col = df_input['违法时间']
    df_date_col['hour'] = df_input['违法时间'].dt.hour
    df_date_col['minute'] = df_input['违法时间'].dt.minute
    a = df_date_col['hour'].value_counts().sort_index()
    plt.subplot(4, 1, 4)
    a.plot(kind='bar', x='Time')
    plt.title("Count of Time")
    plt.show()


# 词云计算
# 输入变量格式为value_counts的结果
from wordcloud import WordCloud


def word_cloud(word_counts):
    counts = {}
    for word in word_counts.index:
        counts[word] = word_counts[word]
    wordcloud = WordCloud(background_color='white', font_path='/System/Library/Fonts/Hiragino Sans GB.ttc', width=1000,
                          height=1000,
                          margin=2)
    wordcloud.generate_from_frequencies(counts)
    # 显示图片
    plt.imshow(wordcloud)

In [ ]:
analyseDF(df_no_yingyun_value)
analyseDF(df_yingyun_value)

### 聚类操作

In [ ]:
# k-mean聚类
# 数据标准化后的聚类计算
from sklearn.cluster import KMeans


def Kmeans(df_input, list_variables):
    X = df_input[list_variables].values
    k_means = KMeans(n_clusters=3, random_state=0).fit(X)

    df_input['label'] = k_means.labels_
    print('聚类的个数')
    print(df_input['label'].value_counts())

    label_pred = k_means.labels_
    centroids = k_means.cluster_centers_
    inertia = k_means.inertia_
    length, dim = X.shape
    df_center = pd.DataFrame(centroids, columns=list_variables)
    display(df_center)
    return df_input


def choose_k(df_input, list_variables):
    SSE = []  # 存放每次结果的误差平方和
    for k in range(1, 9):
        estimator = KMeans(n_clusters=k)  # 构造聚类器
        estimator.fit(df_input[list_variables])
        SSE.append(estimator.inertia_)
    X = range(1, 9)
    plt.xlabel('k')
    plt.ylabel('SSE')
    plt.plot(X, SSE, 'o-')
    plt.show()


In [ ]:
print(df_no_yingyun_level.columns)

非营运车辆结果

In [ ]:
# list_variables = ['age', 'is_local', 'trans_code', 'occur_prob', 'severity', 'score',
#                   'accident', 'counts']

list_variables = ['age', 'occur_prob', 'severity', 'score', 'counts']

In [ ]:
choose_k(df_no_yingyun_level, list_variables)

In [ ]:
cluster_no_yingyun = Kmeans(df_no_yingyun_level, list_variables)

营运车辆结果

In [ ]:
choose_k(df_yingyun_level, list_variables)

In [ ]:
cluster_yingyun = Kmeans(df_yingyun_level, list_variables)

In [ ]:
cluster_no_yingyun.to_csv("/Users/jiarui/Study/交通事故/data/output/df_no_yingyun_cluster_output.csv")
cluster_yingyun.to_csv("/Users/jiarui/Study/交通事故/data/output/df_yingyun_cluster_output.csv")

In [ ]:
def analyse_clusters(df_cluster, df_all_info):
    for i in range(0, 3):
        print('---------------------------k = {}----------------------'.format(i))
        dfi = df_cluster[df_cluster['label'] == i]
        list_ids = list(dfi.index)
        print('共有违法者{}人'.format(len(list_ids)))
        list_analyse = ['当事人_驾驶证号', '机动车_交通方式说明', '违法行为_违法小类', '机动车_号牌号码', '违法时间', '违法行为_违法描述', 'trans_code',
                        'is_local',
                        'age', 'severity']
        df_analyse = df_all_info[df_all_info['当事人_驾驶证号'].isin(list_ids)]
        print(df_analyse['当事人_驾驶证号'].value_counts())
        print('共有违法记录{}条'.format(len(df_analyse)))
        df_analyse['机动车_号牌号码'] = df_analyse['机动车_号牌号码'].str[0:2]
        analyseDF(df_analyse)

非营运

In [ ]:
analyse_clusters(cluster_no_yingyun, df_no_yingyun_value)

营运

In [ ]:
analyse_clusters(cluster_yingyun, df_yingyun_value)

制作词云图

In [ ]:
import jieba

df_no_yingyun_value = pd.read_csv("/Users/jiarui/Study/交通事故/data/output/df_no_yingyun_value.csv")

list_word_input = list(df_no_yingyun_value['机动车_交通方式说明'])


def word_cloud(list_words):
    all_words = []
    for i in list_words:
        words = jieba.lcut(i)
        all_words.append(words)
    print(all_words)


word_cloud(list_word_input)

